# Agentic AI Financial Chatbot

This notebook serves as the main interface for the financial chatbot. Follow the cells below to:
1. Set up the environment
2. Initialize the LLM models
3. Process your financial documents
4. Interact with the chatbot for financial analysis

In [ ]:
# Import required libraries and setup system path
import sys
import os
from pathlib import Path

# Add project root to system path
project_root = str(Path().absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

# Import project modules
from agents.financial_agent import FinancialAgent
from llms.qwen_wrapper import QwenWrapper
from llms.mistral_wrapper import MistralWrapper
from utils.file_loader import FileLoader
from utils.helpers import Helpers
from analysis.budget_calculator import BudgetCalculator
from analysis.trend_analyzer import TrendAnalyzer
from visualizations.chart_generator import ChartGenerator

# Setup logging
Helpers.setup_logging()

# Initialize LLM Models

We'll be using Qwen and Mistral models for our financial chatbot. Follow these steps to set them up:

1. Install the required model packages:
```bash
pip install transformers torch accelerate
```

2. Download the models:
   - Qwen: We'll use Qwen-7B-Chat
   - Mistral: We'll use Mistral-7B-Instruct-v0.1

Note: The models will be downloaded automatically when we initialize them for the first time.

In [ ]:
# Initialize LLM models
from huggingface_hub import login

try:
    # Model IDs from HuggingFace
    QWEN_MODEL_ID = "Qwen/Qwen-7B-Chat"
    MISTRAL_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.1"
    
    print("Initializing Qwen model (this may take a few minutes)...")
    qwen_model = QwenWrapper(QWEN_MODEL_ID)
    print("Qwen model initialized successfully!")
    
    print("\nInitializing Mistral model (this may take a few minutes)...")
    mistral_model = MistralWrapper(MISTRAL_MODEL_ID)
    print("Mistral model initialized successfully!")
    
    # Create financial agent with primary model (using Mistral as primary)
    agent = FinancialAgent(mistral_model)
    print("\nFinancial agent created with Mistral as the primary model!")
    
except Exception as e:
    print(f"Error initializing models: {str(e)}")
    print("Troubleshooting tips:")
    print("1. Make sure you have enough disk space (models are ~15GB each)")
    print("2. Check your internet connection")
    print("3. Ensure you have at least 16GB of RAM available")

# Process Financial Documents

Load and process your financial documents (CSV/PDF). We'll use the sample data for this example.

In [ ]:
# Initialize file loader and load sample data
sample_file = os.path.join(project_root, 'data', 'sample_budget.csv')
try:
    # Process document using the agent's document processing tool
    print("Loading and processing financial data...")
    response = agent.run(f"Process the document {sample_file}")
    print(response)
    
    # Set default monthly income
    print("\nSetting default monthly income...")
    response = agent.run("Set monthly income to 5000")
    print(response)
except Exception as e:
    print(f"Error: {str(e)}")

# Financial Analysis

Analyze the financial data using the budget calculator and trend analyzer.

In [ ]:
# Perform comprehensive analysis
try:
    print("Performing Budget Analysis...")
    response = agent.run("Analyze my budget and provide recommendations")
    print(response)
    
    print("\nAnalyzing Spending Trends...")
    response = agent.run("Show me my spending trends")
    print(response)
except Exception as e:
    print(f"Error in analysis: {str(e)}")

# Visualizations

Generate visual representations of the financial data.

In [ ]:
# Generate visualizations through the agent
try:
    print("Generating financial visualizations...")
    response = agent.run("Generate visualizations of my financial data")
    print(response)
except Exception as e:
    print(f"Error generating visualizations: {str(e)}")

# Chatbot Interaction

The chatbot can now help you with various financial tasks. Here are some example questions and commands:

## Data Management
- "Process my financial document sample_budget.csv"
- "Set my monthly income to 6000"

## Analysis
- "What are my top spending categories?"
- "Analyze my budget and give recommendations"
- "Show me my spending patterns"
- "What's my current savings rate?"
- "Which categories am I overspending in?"

## Visualizations
- "Generate charts for my spending"
- "Show me my monthly spending trends"
- "Create a breakdown of my expenses"

## Combined Analysis
- "Analyze my spending and show me where I can save money"
- "Compare my budget with recommended percentages"
- "Give me a complete financial overview with visualizations"

Type your question below and press Enter. Type 'quit' to exit.

In [ ]:
def suggest_commands():
    """Suggest some commands based on what hasn't been done yet"""
    suggestions = []
    
    if not agent.current_data:
        suggestions.append("- Process a financial document first: 'Process my financial document sample_budget.csv'")
    if not agent.monthly_income:
        suggestions.append("- Set your monthly income: 'Set my monthly income to 5000'")
    if agent.current_data and agent.monthly_income:
        suggestions.extend([
            "- Analyze your budget: 'Analyze my budget'",
            "- Generate visualizations: 'Generate charts for my spending'",
            "- Get spending insights: 'Show me my spending patterns'"
        ])
    
    return suggestions

def chat_with_bot():
    print("Financial Chatbot: Hello! I'm here to help you with your financial analysis.")
    print("Type 'quit' to exit the chat or 'help' for suggestions.")
    
    while True:
        user_input = input("\nYou: ").strip().lower()
        
        if user_input == 'quit':
            print("\nFinancial Chatbot: Goodbye! Have a great day!")
            break
            
        if user_input == 'help':
            suggestions = suggest_commands()
            if suggestions:
                print("\nFinancial Chatbot: Here are some suggested commands:")
                print("\n".join(suggestions))
            else:
                print("\nFinancial Chatbot: You're all set! Feel free to ask any financial questions.")
            continue
            
        try:
            response = agent.run(user_input)
            print(f"\nFinancial Chatbot: {response}")
        except Exception as e:
            print(f"\nFinancial Chatbot: Sorry, I encountered an error: {str(e)}")
            print("\nTry asking for 'help' to see suggested commands.")

# Start the chat
chat_with_bot()